In [1]:
import glob
import json
import ipaddress

In [2]:
def read_hosts(path):
    nws = set()
    for h in glob.glob(path):
        with open(h) as fp:
            content = json.load(fp)
            nws.add(ipaddress.ip_network(content['hostInterfaces']['eth0']['prefix'], strict=False))
    return nws

In [3]:
def find_in_lines(needle, search):
    diff_specs = []
    for i, l in enumerate(search.splitlines() if type(search) is str else search):
        # print(needle.__str__())
        if needle.__str__() in l:
            diff_specs.append(l)
            # print(f"Found {needle} in line {i}:", l)
    return diff_specs

In [4]:
from pybatfish.client.session import Session
def find_config_prefixes(path):
  bf = Session("127.0.0.1", 9997, 9996)
  bf.set_network("test")
  bf.init_snapshot(path, name="test", overwrite=True)

  # Get the prefixes from the config
  # interfaces = bf.q.interfaceProperties().answer().frame()
  # print(dir(bf.q))
  dat = set()
  interfaces = bf.q.interfaceProperties().answer().frame()

  for pfxs in interfaces['All_Prefixes']:
    # print(pfxs)
    for pfx in pfxs:
      dat.add(ipaddress.ip_network(pfx, strict=False))

  return dat
  # print(interfaces)
  # print(path)
  

In [15]:
NETWORKS_DIR = "/home/yongting/research/configs/config2spec-confmask/scenarios/confmask/NetworkA-CCNP"
networks = glob.glob(NETWORKS_DIR + "/*")
networks.remove(f"{NETWORKS_DIR}/origin")
networks.insert(0, f"{NETWORKS_DIR}/origin")
print("network_name", "specs#","kept#","false_positive#","false_negative#","#false_negative_fake_dsts","#false_negative_fake_dsts","#hosts", sep='\t')
orig_config_prefixes = set()
orig_host_prefixes = set()
import os

for nw in networks:
    nw_name = nw.split("/")[-1]
    find_config_prefixes(nw)
    if "strawman" in nw_name:
        continue
    # read host prefixes
    if nw_name == "origin":
        orig_host_prefixes = read_hosts(nw + "/hosts/host*.json")
        orig_config_prefixes = find_config_prefixes(nw)
        # print(orig_host_prefixes)
        # print(orig_config_prefixes)
    nw_host_pfxs = read_hosts(nw + "/hosts/host*.json")
    nw_config_prefixes = find_config_prefixes(nw)
    if not os.path.exists(NETWORKS_DIR + f"/{nw_name}/policies-holds.csv"):
        continue
    with open(NETWORKS_DIR + f"/{nw_name}/policies-holds.csv", 'r') as fp:
        nw_content = set(fp.read().__str__().splitlines()[1:])
    with open(NETWORKS_DIR + f"/{nw_name}/policies-holds-not.csv", 'r') as fp:
        nw_content_holds_not = set(fp.read().__str__().splitlines()[1:])
        nw_content = nw_content.union(nw_content_holds_not)
    with open(NETWORKS_DIR + f"/origin/policies-holds.csv", 'r') as fp:
        origin_content = set(fp.read().__str__().splitlines()[1:])
    with open(NETWORKS_DIR + f"/origin/policies-holds-not.csv", 'r') as fp:
        origin_content_holds_not = set(fp.read().__str__().splitlines()[1:])
        origin_content = origin_content.union(origin_content_holds_not)
    # print(len(nw_content), len(origin_content))
    origin_policies = origin_content
    nw_policies = nw_content
    fpos = nw_policies - origin_policies
    fneg = origin_policies - nw_policies
    kept = origin_policies & nw_policies

    # fpos and fneg introduced by fake hosts
    fneg_host_count = 0
    fpos_host_count = 0
    for pfx in (nw_host_pfxs - orig_host_prefixes).union(nw_config_prefixes - orig_config_prefixes):
        diff_specs = find_in_lines(pfx, list(fneg))
        fneg_host_count += len(diff_specs)
        diff_specs = find_in_lines(pfx, list(fpos))
        # print(diff_specs)
        fpos_host_count += len(diff_specs)

    # fpos and fneg introduced by fake links
    # fneg_link_count = 0
    # fpos_link_count = 0
    # for pfx in nw_config_prefixes - orig_config_prefixes:
    #     diff_specs = find_in_lines(pfx, list(fneg))
    #     fneg_link_count += len(diff_specs)
    #     diff_specs = find_in_lines(pfx, list(fpos))
    #     # print(diff_specs)
    #     fpos_link_count += len(diff_specs)
    
    # fpos_fake_dst_count = fpos_link_count + fpos_host_count
    # fneg_fake_dst_count = fneg_link_count + fneg_host_count

    # with open(NETWORKS_DIR + f"/{nw_name}/policies.csv") as fp:
    #     specs_count = len(fp.read().splitlines()) - 1
    print(nw_name, len(nw_policies), len(kept), len(fpos), len(fneg), fneg_host_count, fpos_host_count, len(nw_host_pfxs), sep="\t")
    # break

network_name	specs#	kept#	false_positive#	false_negative#	#false_negative_fake_dsts	#false_negative_fake_dsts	#hosts
origin	82	82	0	0	0	0	8
anonym-network-k6-w-noise	229	54	175	35	0	251	32
anonym-network-k6-4-w-noise	371	56	315	29	0	250	32
anonym-network-k2-2-w-noise	460	67	393	16	0	87	16
anonym-network-k6-6-w-noise	692	67	625	15	0	432	48
anonym-network-k2-w-noise	745	72	673	10	0	74	16
anonym-network-k10-2-w-noise	811	72	739	10	0	76	16
anonym-network-k6-2-w-noise	857	72	785	10	0	74	16
anonym-network-k10-w-noise	908	74	834	8	0	73	16


In [6]:
def get_trace_accepted_count(traces):
    accepted_count = 0
    # print(traces[0])
    for t in traces[0]:
        if t.disposition in ["ACCEPTED", "EXITS_NETWORK"]:
            accepted_count += 1
    return accepted_count
# print(accepted_count)
# a.Traces[0][0].dict()

In [10]:
# bf = Session("127.0.0.1", 10007, 9996)

In [8]:
NETWORKS_DIR = "/home/yongting/research/configs/config2spec-confmask/scenarios/confmask/fattree04-ospf"
from pybatfish.datamodel.flow import PathConstraints
from pybatfish.client.session import Session, HeaderConstraints
import glob
import threading
networks = glob.glob(NETWORKS_DIR + "/*")
networks.remove(f"{NETWORKS_DIR}/origin")
networks.insert(0, f"{NETWORKS_DIR}/origin")
a = None
bf = None
results = []

def trace_tasks(bf, specs, idx):
    i = 0
    specs_count = len(specs)
    holds_specs = []
    holds_not_specs = []
    for spec in specs:
        print(f" - Proc {idx}: {i}/{specs_count}")
        i += 1
        policy_type = spec.split(",")[0]
        src_name = spec.split(",")[3]
        dst_name = spec.split(",")[-2].split(':')[0][1:]
        dst_pfx = spec.split(",")[1]
        result = bf.q.reachability(pathConstraints=PathConstraints(startLocation = f'{src_name}'), headers=HeaderConstraints(dstIps=dst_pfx, srcIps='0.0.0.0/0'), actions='SUCCESS').answer().frame()

        policy_holds = False

        if policy_type == "PolicyType.Waypoint":
            accepted = get_trace_accepted_count(result.Traces)
            if accepted > 0:
                policy_holds = True
            pass

        if policy_type == "PolicyType.LoadBalancingSimple":
            accepted = get_trace_accepted_count(result.Traces)
            if accepted == int(spec.split(",")[2]):
                policy_holds = True
            pass

        if policy_type == "PolicyType.Reachability":
            accepted = get_trace_accepted_count(result.Traces)
            if accepted > 0:
                policy_holds = True
            pass

        if policy_type == "PolicyType.Isolation":
            accepted = get_trace_accepted_count(result.Traces)
            if accepted == 0:
                policy_holds = True
            pass

        if policy_holds:
            # holds_count += 1
            spec = spec.replace("HOLDSNOT", "HOLDS")
            holds_specs.append(spec)
        else:
            # holds_not_count += 1
            if "HOLDSNOT" not in spec:
                spec = spec.replace("HOLDS", "HOLDSNOT")
            print("NotHolds", spec)
            holds_not_specs.append(spec)
    results[idx] = (holds_specs, holds_not_specs)

for nw in networks[1:]:
    nw_name = nw.split("/")[-1]
    print(nw_name)

    bf = Session("127.0.0.1", 9997, 9996)
    bf.set_network("test")
    bf.init_snapshot(nw, name="test", overwrite=True)

    # verify specs one by one
    with open(nw + "/policies.csv", 'r') as fp:
        nw_specs_content = fp.read().__str__().splitlines()

    # specs_count = 0
    # holds_count = 0
    # holds_not_count = 0
    holds_specs = [nw_specs_content[0]]
    holds_not_specs = [nw_specs_content[0]]
    # specs_count = len(nw_specs_content) - 1

    thread_count = 10
    i = 0
    # divide specs into `thread_count` parts
    ts = []
    for idx in range(0, len(nw_specs_content[1:]), len(nw_specs_content[1:]) // (thread_count + 1)):
        specs = nw_specs_content[1:][idx:idx+len(nw_specs_content[1:]) // (thread_count + 1) - 1]
        # print(len(specs))
        # print(idx,idx+len(nw_specs_content[1:]) // (thread_count + 1) - 1)
        results.append(None)
        t = threading.Thread(target=trace_tasks, args=(bf, specs, i))
        ts.append(t)
        t.start()
        thread_count += 1
        i += 1
    
    for t in ts:
        t.join()


    for task_result in results:
        holds_specs.extend(task_result[0])
        holds_not_specs.extend(task_result[1])
    
    with open(f"{nw}/policies-holds.csv", "w+") as fp:
        fp.write("\n".join(holds_specs))
    with open(f"{nw}/policies-holds-not.csv", "w+") as fp:
        fp.write("\n".join(holds_not_specs))


anonym-network-k2
 - Proc 0: 0/231
 - Proc 1: 0/212
 - Proc 2: 0/195
 - Proc 3: 0/181
 - Proc 4: 0/169
 - Proc 5: 0/159
 - Proc 6: 0/149
 - Proc 7: 0/141
 - Proc 8: 0/133
 - Proc 9: 0/127
 - Proc 10: 0/120
 - Proc 11: 0/8
 - Proc 0: 1/231 - Proc 1: 1/212

 - Proc 2: 1/195
 - Proc 5: 1/159
 - Proc 3: 1/181
 - Proc 6: 1/149
 - Proc 7: 1/141
 - Proc 8: 1/133
 - Proc 9: 1/127


KeyboardInterrupt: 

 - Proc 11: 1/8
 - Proc 10: 1/120
 - Proc 4: 1/169
 - Proc 0: 2/231
 - Proc 1: 2/212
 - Proc 2: 2/195
 - Proc 6: 2/149
 - Proc 3: 2/181
 - Proc 5: 2/159
 - Proc 11: 2/8
 - Proc 8: 2/133
 - Proc 7: 2/141
 - Proc 9: 2/127
 - Proc 10: 2/120
 - Proc 4: 2/169
 - Proc 0: 3/231
 - Proc 6: 3/149
 - Proc 3: 3/181
 - Proc 1: 3/212
 - Proc 2: 3/195
 - Proc 11: 3/8
 - Proc 8: 3/133
 - Proc 7: 3/141
 - Proc 5: 3/159
 - Proc 10: 3/120
 - Proc 4: 3/169
 - Proc 0: 4/231
 - Proc 6: 4/149
 - Proc 3: 4/181
 - Proc 9: 3/127
 - Proc 1: 4/212
 - Proc 2: 4/195
 - Proc 11: 4/8
 - Proc 10: 4/120
 - Proc 8: 4/133
 - Proc 7: 4/141
 - Proc 3: 5/181
 - Proc 5: 4/159
 - Proc 4: 4/169
 - Proc 1: 5/212
 - Proc 2: 5/195
 - Proc 6: 5/149
 - Proc 9: 4/127


Exception in thread Exception in threading.excepthook:
Exception ignored in thread started by: <bound method Thread._bootstrap of <Thread(Thread-28 (trace_tasks), stopped 140120816608832)>>
Traceback (most recent call last):
  File "/home/yongting/anaconda3/envs/net2text311/lib/python3.11/threading.py", line 995, in _bootstrap


 - Proc 0: 5/231 - Proc 11: 5/8
 - Proc 10: 5/120


      File "/home/yongting/anaconda3/envs/net2text311/lib/python3.11/threading.py", line 1040, in _bootstrap_inner
    self._invoke_excepthook(self)
  File "/home/yongting/anaconda3/envs/net2text311/lib/python3.11/threading.py", line 1352, in invoke_excepthook
    local_print("Exception in threading.excepthook:",
  File "/home/yongting/anaconda3/envs/net2text311/lib/python3.11/site-packages/ipykernel/iostream.py", line 604, in flush
    self.pub_thread.schedule(self._flush)
  File "/home/yongting/anaconda3/envs/net2text311/lib/python3.11/site-packages/ipykernel/iostream.py", line 267, in schedule
    self._event_pipe.send(b"")
  File "/home/yongting/anaconda3/envs/net2text311/lib/python3.11/site-packages/zmq/sugar/socket.py", line 696, in send
    return super().send(data, flags=flags, copy=copy, track=track)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "zmq/backend/cython/socket.pyx", line 742, in zmq.backend.cython.socket.Socket.send
  File "zmq/backend/c

 - Proc 8: 5/133
 - Proc 4: 5/169
 - Proc 7: 5/141
 - Proc 2: 6/195
 - Proc 3: 6/181
 - Proc 1: 6/212
 - Proc 5: 5/159
 - Proc 9: 5/127
 - Proc 6: 6/149
 - Proc 11: 6/8
 - Proc 4: 6/169
 - Proc 10: 6/120
 - Proc 8: 6/133
 - Proc 7: 6/141
 - Proc 3: 7/181
 - Proc 2: 7/195
 - Proc 9: 6/127
 - Proc 5: 6/159
 - Proc 1: 7/212
 - Proc 6: 7/149
 - Proc 11: 7/8
 - Proc 4: 7/169
 - Proc 10: 7/120
 - Proc 8: 7/133
 - Proc 7: 7/141
 - Proc 3: 8/181
 - Proc 2: 8/195
 - Proc 9: 7/127
 - Proc 5: 7/159
 - Proc 1: 8/212
 - Proc 6: 8/149
 - Proc 7: 8/141
 - Proc 8: 8/133
 - Proc 3: 9/181
 - Proc 2: 9/195
 - Proc 4: 8/169
 - Proc 10: 8/120
 - Proc 1: 9/212
 - Proc 5: 8/159
 - Proc 9: 8/127
 - Proc 7: 9/141
 - Proc 8: 9/133
 - Proc 3: 10/181
 - Proc 6: 9/149
 - Proc 4: 9/169
 - Proc 2: 10/195
 - Proc 10: 9/120
 - Proc 1: 10/212
 - Proc 5: 9/159
 - Proc 7: 10/141
 - Proc 8: 10/133
 - Proc 3: 11/181
 - Proc 6: 10/149
 - Proc 9: 9/127
 - Proc 4: 10/169
 - Proc 10: 10/120
 - Proc 2: 11/195
 - Proc 1: 11/212


Exception in thread Thread-36 (trace_tasks):
Traceback (most recent call last):
  File "/home/yongting/anaconda3/envs/net2text311/lib/python3.11/site-packages/pybatfish/client/restv2helper.py", line 657, in _check_response_status
    response.raise_for_status()
  File "/home/yongting/anaconda3/envs/net2text311/lib/python3.11/site-packages/requests/models.py", line 1021, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 404 Client Error: Not Found for url: http://127.0.0.1:9996/v2/networks/test/questions/__reachability_d5afe537-71e4-4f3a-8612-e44cfe4b767e/answer?snapshot=test

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/yongting/anaconda3/envs/net2text311/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/home/yongting/anaconda3/envs/net2text311/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 761, in run_closure
    _th

In [72]:
a.Traces[0]

ListWrapper([((ORIGINATED(default), FORWARDED(Forwarded out interface: GigabitEthernet0/1 with resolved next-hop IP: 10.2.0.0, Routes: [ospfIA (Network: 32.249.160.0/24, Next Hop: interface GigabitEthernet0/1 ip 10.2.0.0)]), TRANSMITTED(GigabitEthernet0/1)), (RECEIVED(GigabitEthernet0/1), FORWARDED(Forwarded out interface: GigabitEthernet0/2 with resolved next-hop IP: 10.2.1.1, Routes: [ospfIA (Network: 32.249.160.0/24, Next Hop: interface GigabitEthernet0/2 ip 10.2.1.1)]), TRANSMITTED(GigabitEthernet0/2)), (RECEIVED(GigabitEthernet0/1), FORWARDED(Forwarded out interface: GigabitEthernet0/4 with resolved next-hop IP: 20.1.1.3, Routes: [ospf (Network: 32.249.160.0/24, Next Hop: interface GigabitEthernet0/4 ip 20.1.1.3)]), TRANSMITTED(GigabitEthernet0/4)), (RECEIVED(GigabitEthernet0/2), FORWARDED(Forwarded out interface: GigabitEthernet50/0, Routes: [connected (Network: 32.249.160.0/24, Next Hop: interface GigabitEthernet50/0)]), TRANSMITTED(GigabitEthernet50/0), EXITS_NETWORK(Output Interface: GigabitEthernet50/0, Resolved Next Hop IP: 32.249.160.0))), ((ORIGINATED(default), FORWARDED(Forwarded out interface: GigabitEthernet0/2 with resolved next-hop IP: 10.3.0.0, Routes: [ospfIA (Network: 32.249.160.0/24, Next Hop: interface GigabitEthernet0/2 ip 10.3.0.0)]), TRANSMITTED(GigabitEthernet0/2)), (RECEIVED(GigabitEthernet0/1), FORWARDED(Forwarded out interface: GigabitEthernet0/2 with resolved next-hop IP: 10.3.1.1, Routes: [ospfIA (Network: 32.249.160.0/24, Next Hop: interface GigabitEthernet0/2 ip 10.3.1.1)]), TRANSMITTED(GigabitEthernet0/2)), (RECEIVED(GigabitEthernet0/2), FORWARDED(Forwarded out interface: GigabitEthernet0/4 with resolved next-hop IP: 20.1.1.3, Routes: [ospf (Network: 32.249.160.0/24, Next Hop: interface GigabitEthernet0/4 ip 20.1.1.3)]), TRANSMITTED(GigabitEthernet0/4)), (RECEIVED(GigabitEthernet0/2), FORWARDED(Forwarded out interface: GigabitEthernet50/0, Routes: [connected (Network: 32.249.160.0/24, Next Hop: interface GigabitEthernet50/0)]), TRANSMITTED(GigabitEthernet50/0), EXITS_NETWORK(Output Interface: GigabitEthernet50/0, Resolved Next Hop IP: 32.249.160.0)))])

In [64]:
bf.q.routes(network="32.11.137.0/24").answer().frame()

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag
0,agg0-0,default,32.11.137.0/24,interface GigabitEthernet0/1 ip 10.0.0.0,10.0.0.0,GigabitEthernet0/1,ospfIA,7,110,None
1,agg0-0,default,32.11.137.0/24,interface GigabitEthernet0/2 ip 10.1.0.0,10.1.0.0,GigabitEthernet0/2,ospfIA,7,110,None
2,agg0-1,default,32.11.137.0/24,interface GigabitEthernet0/1 ip 10.2.0.0,10.2.0.0,GigabitEthernet0/1,ospfIA,7,110,None
3,agg0-1,default,32.11.137.0/24,interface GigabitEthernet0/2 ip 10.3.0.0,10.3.0.0,GigabitEthernet0/2,ospfIA,7,110,None
4,agg1-0,default,32.11.137.0/24,interface GigabitEthernet0/1 ip 10.0.1.0,10.0.1.0,GigabitEthernet0/1,ospfIA,7,110,None
5,agg1-0,default,32.11.137.0/24,interface GigabitEthernet0/2 ip 10.1.1.0,10.1.1.0,GigabitEthernet0/2,ospfIA,7,110,None
6,agg1-1,default,32.11.137.0/24,interface GigabitEthernet0/1 ip 10.2.1.0,10.2.1.0,GigabitEthernet0/1,ospfIA,7,110,None
7,agg1-1,default,32.11.137.0/24,interface GigabitEthernet0/2 ip 10.3.1.0,10.3.1.0,GigabitEthernet0/2,ospfIA,7,110,None
8,agg2-0,default,32.11.137.0/24,interface GigabitEthernet0/4 ip 20.2.0.3,20.2.0.3,GigabitEthernet0/4,ospf,3,110,None
9,agg2-1,default,32.11.137.0/24,interface GigabitEthernet0/4 ip 20.2.1.3,20.2.1.3,GigabitEthernet0/4,ospf,3,110,None
